# How-to: Predict and update URLs for OPD data

## Environment pre-reqs
To install all required packages from `requirements.txt` in this folder, run:

In [ ]:
pip install -r requirements.txt

Or, if you are using conda, you can create a new environment and install the requirements with:

In [ ]:
conda create -n opd-env python=3.11
conda activate opd-env
pip install -r requirements.txt

## Prediction functions

In [2]:
# import local script
import prediction_funcs

## Find additional datasets from predictable URLs

#### `try_url_years` function
This function can be used to test for additional datasets from predictable URLs. Before adding a URL to the OPD Source Table, it will also assure it's not already there. If it is, it will retest if it returns data and appropriately updated the "last_coverage_check" field or remove deprecated URLs.

Fields required to add a new source to the OPD Source Table can be passed into the function in the "spreadsheet_fields" dict arg, as well as below additional fields as available.

**Additional spreadsheet fields:**
"Description", "source_url", "readme", "date_field", "agency_originated", "supplying_entity", "agency_field", "min_version", "query"

The `try_urls_years` function auto-fills the spreadsheet's "URL", "Year","last_coverage_check", "coverage_start", and "coverage_end".

**Function's args:**
- url: str. The URL to modify. *Required*.
- spreadsheet_fields: dict with fields to use for checking validity and adding to OPD_Source_table. *Required*.
- n_years: int or range. If int, tries n_years forward (or backward if forward=False). If range, uses as years to try. Optional, defaults to 5.
- forward: If n_years is int, direction to try. Optional, defaults to True.
- year_slice: tuple of (start, end) to slice the URL for the year. Optional, finds the first 4-digit year in the URL.
- verbose: If True, prints progress messages. Optional, defaults to False.

#### Helpers
Current TableTypes:

In [13]:
opd = pd.read_csv("../../opd_source_table.csv")
opd["TableType"].unique()

array(['ARRESTS', 'CALLS FOR SERVICE', 'INCIDENTS', 'EMPLOYEE', 'STOPS',
       'OFFICER-INVOLVED SHOOTINGS', 'POINTING WEAPON',
       'TRAFFIC CITATIONS', 'USE OF FORCE', 'CRASHES',
       'OFFICER-INVOLVED SHOOTINGS - INCIDENTS',
       'OFFICER-INVOLVED SHOOTINGS - OFFICERS',
       'OFFICER-INVOLVED SHOOTINGS - SUBJECTS', 'TRAFFIC STOPS',
       'DEATHS IN CUSTODY', 'USE OF FORCE - INCIDENTS',
       'USE OF FORCE - SUBJECTS/OFFICERS', 'STOPS - INCIDENTS',
       'STOPS - SUBJECTS', 'CITATIONS', 'COMPLAINTS',
       'USE OF FORCE - ADDITIONAL', 'VEHICLE PURSUITS',
       'COMPLAINTS - ALLEGATIONS', 'COMPLAINTS - BACKGROUND',
       'COMPLAINTS - BODY WORN CAMERA', 'COMPLAINTS - SUBJECTS',
       'CRASHES - INCIDENTS', 'CRASHES - SUBJECTS',
       'USE OF FORCE - OFFICERS', 'USE OF FORCE - SUBJECTS', 'LAWSUITS',
       'COMPLAINTS - OFFICERS', 'PEDESTRIAN STOPS', 'FIELD CONTACTS',
       'TRAFFIC WARNINGS', 'CRASHES - NONMOTORIST',
       'DISCIPLINARY RECORDS', 'COMPLAINTS - PENAL

### The function with args template for use :)

In [ ]:
url = "https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2013_csv/FeatureServer/0"
spreadsheet_fields = {
    # required fields
    "State": "Maryland",
    "SourceName": "Baltimore",
    "Agency": "BPD",
    "AgencyFull": "Baltimore Police Department",
    "TableType": "CALLS FOR SERVICE",
    "DataType": "ArcGIS"
    # below required if "DataType" = "Carto", "Ckan", or "Socrata"
    # , dataset_id": "example-dataset-id" # this is unlikely to be the same for multiple years, but leaving for completeness and future improvements
    # additional fields as needed
    , "source_url": "https://services1.arcgis.com/UWYHeuuJISiGmgXx/ArcGIS/rest/services/911_2013_2022/FeatureServer"
    , "agency_originated": "yes" 
}

prediction_funcs.try_url_years(url, n_years=10, verbose=True, spreadsheet_fields=spreadsheet_fields)

https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2014_csv/FeatureServer/0: not valid. Skipping.
https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2015_csv/FeatureServer/0: not valid. Skipping.
https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2016_csv/FeatureServer/0: not valid. Skipping.
https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2017_csv/FeatureServer/0 already in spreadsheet and valid. Updated last_coverage_check.
https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2018_csv/FeatureServer/0 already in spreadsheet and valid. Updated last_coverage_check.
https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2019_csv/FeatureServer/0 already in spreadsheet and valid. Updated last_coverage_check.
Valid URL found: https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Servi

## Check all OPD Sources for new predictable URLs

#### `auto_update_sources` function
This function does a lot more of the leg work for you, by simply checking the existing sources for predictable URLs that may have new datasets available. It does so by automatically looking for where the "last_coverage_check" date is last year or older, but you can use the `outdated_days` arg to check more recently checked sources.

Note it only checks forward, as it first groups by fields 'State', 'SourceName', 'AgencyFull', and 'TableType' as a composite key, and gets the max Year to test forward from, stopping at current year.

The function also assumes all other fields for the new URLs, with the exception of date-related fields, can be duplicated from the latest record. Please confirm any descriptions, readmes, etc, are still relevant. 

In [3]:
prediction_funcs.auto_update_sources(
    # outdated_days=30 # unspecified defaults to last year
    # ,
    # verbose=True
)

Checked 15 sources for new URLs, found 4 new sources.
